In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/TableNet-pytorch/Training

/content/drive/MyDrive/TableNet-pytorch/Training


In [8]:
import pandas as pd
testdf = pd.read_csv('../processed_data_v2.csv')
testdf.head()

,img_path,table_mask,col_mask,original_height,original_width,hasTable,table_count,col_count,table_bboxes,col_bboxes
0,marmot_processed\image_v2\10.1.1.1.2000_4.jpg,marmot_processed\table_mask\10.1.1.1.2000_4_ta...,marmot_processed\col_mask\10.1.1.1.2000_4_col_...,1008,768,0,0,0,[],[]
1,marmot_processed\image_v2\10.1.1.1.2004_4.jpg,marmot_processed\table_mask\10.1.1.1.2004_4_ta...,marmot_processed\col_mask\10.1.1.1.2004_4_col_...,1123,793,0,0,0,[],[]
2,marmot_processed\image_v2\10.1.1.1.2004_5.jpg,marmot_processed\table_mask\10.1.1.1.2004_5_ta...,marmot_processed\col_mask\10.1.1.1.2004_5_col_...,1123,793,0,0,0,[],[]
3,marmot_processed\image_v2\10.1.1.1.2005_12.jpg,marmot_processed\table_mask\10.1.1.1.2005_12_t...,marmot_processed\col_mask\10.1.1.1.2005_12_col...,1056,816,0,0,0,[],[]
4,marmot_processed\image_v2\10.1.1.1.2005_13.jpg,marmot_processed\table_mask\10.1.1.1.2005_13_t...,marmot_processed\col_mask\10.1.1.1.2005_13_col...,1056,816,0,0,0,[],[]


In [10]:
%pip install efficientnet_pytorch pytorch_model_summary

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=cd483b114ae3cf75760e617c3b9a7c114655ee079c98cd9751f279e3f10e141b
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [11]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import config
import os
from utils import (
    get_data_loaders,
    load_checkpoint,
    save_checkpoint,
    display_metrics,
    write_summary,
    compute_metrics,
    seed_all
    )
from loss import TableNetLoss
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from model import TableNet
from torch.utils.tensorboard import SummaryWriter
from pytorch_model_summary import summary


In [12]:
import sys
import warnings
warnings.filterwarnings("ignore")

In [13]:
def train_on_epoch(data_loader, model, optimizer, loss, scaler, threshold = 0.5):
    print("in train_on_epoch")
    # metrics = 0
    combined_loss = []
    table_loss, table_acc, table_precision, table_recall, table_f1 = [],[],[],[],[]
    col_loss, col_acc, col_precision, col_recall, col_f1 = [],[],[],[],[]

    loop = tqdm(data_loader, leave=True)
    # print("THIS IS LOOP",len(list(data_loader)), loop, "THIS IS LOOP")

    for batch_idx, img_dict in enumerate(loop):
        image           = img_dict["image"].to(config.DEVICE)
        table_image     = img_dict["table_image"].to(config.DEVICE)
        column_image     = img_dict["column_image"].to(config.DEVICE)

        with torch.cuda.amp.autocast():
            table_out, column_out  = model(image)
            t_loss, c_loss = loss(table_out, table_image, column_out, column_image)
        table_loss.append(t_loss.item())
        col_loss.append(c_loss.item())
        combined_loss.append((t_loss + c_loss).item())

        #backward
        optimizer.zero_grad()
        scaler.scale(t_loss + c_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        mean_loss = sum(combined_loss) / len(combined_loss)
        loop.set_postfix(loss=mean_loss)

        cal_metrics_table = compute_metrics(table_image, table_out, threshold)
        cal_metrics_col = compute_metrics(column_image, column_out, threshold)

        table_f1.append(cal_metrics_table['f1'])
        table_precision.append(cal_metrics_table['precision'])
        table_acc.append(cal_metrics_table['acc'])
        table_recall.append(cal_metrics_table['recall'])
        col_f1.append(cal_metrics_col['f1'])
        col_acc.append(cal_metrics_col['acc'])
        col_precision.append(cal_metrics_col['precision'])
        col_recall.append(cal_metrics_col['recall'])

        metrics = {
        'combined_loss': np.mean(combined_loss),
        'table_loss': np.mean(table_loss),
        'column_loss': np.mean(col_loss),
        'table_f1': np.mean(table_f1),
        'col_f1': np.mean(col_f1),
        'table_precision': np.mean(table_precision),
        'col_precision': np.mean(col_precision),
        'table_recall': np.mean(table_recall),
        'col_recall': np.mean(col_recall),
        'table_acc': np.mean(table_acc),
        'col_acc': np.mean(col_acc)
        }

    return metrics



In [14]:
def test_on_epoch(data_loader, model, loss, threshold = 0.5, device = config.DEVICE):

    combined_loss = []
    table_loss, table_acc, table_precision, table_recall, table_f1 = [],[],[],[],[]
    col_loss, col_acc, col_precision, col_recall, col_f1 = [],[],[],[],[]

    model.eval()
    with torch.no_grad():
        loop = tqdm(data_loader, leave=True)

        for batch_idx, img_dict in enumerate(loop):
            image           = img_dict["image"].to(device)
            table_image     = img_dict["table_image"].to(device)
            column_image     = img_dict["column_image"].to(device)

            with torch.cuda.amp.autocast():
                table_out, column_out  = model(image)

                t_loss, c_loss = loss(table_out, table_image, column_out, column_image)
            table_loss.append(t_loss.item())
            col_loss.append(c_loss.item())
            combined_loss.append((t_loss + c_loss).item())

            mean_loss = sum(combined_loss) / len(combined_loss)
            loop.set_postfix(loss=mean_loss)

            #cal_metrics = compute_metrics(table_image, column_image, table_out, column_out, threshold)

            cal_metrics_table = compute_metrics(table_image, table_out, threshold)
            cal_metrics_col = compute_metrics(column_image, column_out, threshold)


            table_f1.append(cal_metrics_table['f1'])
            table_precision.append(cal_metrics_table['precision'])
            table_acc.append(cal_metrics_table['acc'])
            table_recall.append(cal_metrics_table['recall'])
            col_f1.append(cal_metrics_col['f1'])
            col_acc.append(cal_metrics_col['acc'])
            col_precision.append(cal_metrics_col['precision'])
            col_recall.append(cal_metrics_col['recall'])



    metrics = {
        'combined_loss': np.mean(combined_loss),
        'table_loss': np.mean(table_loss),
        'column_loss': np.mean(col_loss),
        'table_f1': np.mean(table_f1),
        'col_f1': np.mean(col_f1),
        'table_precision': np.mean(table_precision),
        'col_precision': np.mean(col_precision),
        'table_recall': np.mean(table_recall),
        'col_recall': np.mean(col_recall),
        'table_acc': np.mean(table_acc),
        'col_acc': np.mean(col_acc)
    }

    model.train()
    return metrics


In [15]:
if __name__ == '__main__':
    print("1")
    seed_all(SEED_VALUE = config.SEED)
    checkpoint_name = 'densenet_config_4_model_checkpoint.pth.tar'
    model = TableNet(encoder = 'densenet', use_pretrained_model = True, basemodel_requires_grad = True)
    print("2")


    print("Model Architecture and Trainable Paramerters")
    print("="*50)
    print("3")
    print(summary(model, torch.zeros((1, 3, 1024, 1024)), show_input=False, show_hierarchical=True))
    print("4")
    print(config.DEVICE)

    model = model.to(config.DEVICE)
    print("5")
    optimizer = optim.Adam(
        model.parameters(),
        lr = config.LEARNING_RATE,
        weight_decay = config.WEIGHT_DECAY,
        )
    loss = TableNetLoss()
    print("6")
    scaler = torch.cuda.amp.GradScaler()
    print("7")
    train_loader, test_loader = get_data_loaders(data_path = config.DATAPATH)
    print("8")

    nl = '\n'

    #load checkpoint
    if os.path.exists(checkpoint_name):
        last_epoch, tr_metrics,te_metrics = load_checkpoint(torch.load(checkpoint_name), model)
        last_table_f1 = te_metrics['table_f1']
        last_col_f1 = te_metrics['col_f1']

        print("Loading Checkpoint")
        display_metrics(last_epoch, tr_metrics,te_metrics)
        print("9")
        print()
    else:
        last_epoch = 0
        last_table_f1 = 0.
        last_col_f1 = 0.

    #Train Network
    print("Training Model123\n")
    print("9")
    writer = SummaryWriter(
            f"runs/TableNet/densenet/config_4_batch_{config.BATCH_SIZE}_LR_{config.LEARNING_RATE}_encoder_train"
        )

    #for early stopping
    i = 0
    print("10")

    for epoch in range(last_epoch + 1,config.EPOCHS):
    # for epoch in range(last_epoch + 1,3):
        print("11")
        print("="*30)
        start = time.time()

        tr_metrics = train_on_epoch(train_loader, model, optimizer, loss, scaler, 0.5)
        print("12")
        te_metrics = test_on_epoch(test_loader, model, loss, threshold = 0.5)

        write_summary(writer, tr_metrics, te_metrics, epoch)

        end = time.time()

        display_metrics(epoch, tr_metrics,te_metrics)

        if last_table_f1 < te_metrics['table_f1'] or last_col_f1 < te_metrics['col_f1']:

            #i = 0
            last_table_f1 = te_metrics['table_f1']
            last_col_f1 = te_metrics['col_f1']

            checkpoint = {'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'train_metrics': tr_metrics, 'test_metrics': te_metrics}
            save_checkpoint(checkpoint, checkpoint_name)
        else:
           i += 1

        if i == 12:
           print("Early Stopping")
           break


1


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 151MB/s]


2
Model Architecture and Trainable Paramerters
3
-------------------------------------------------------------------------------------------------------------
      Layer (type)                                              Output Shape         Param #     Tr. Param #
        DenseNet-1     [1, 256, 64, 64], [1, 512, 32, 32], [1, 1024, 32, 32]       6,951,808       6,951,808
          Conv2d-2                                          [1, 256, 32, 32]         262,400         262,400
            ReLU-3                                          [1, 256, 32, 32]               0               0
         Dropout-4                                          [1, 256, 32, 32]               0               0
          Conv2d-5                                          [1, 256, 32, 32]          65,792          65,792
            ReLU-6                                          [1, 256, 32, 32]               0               0
         Dropout-7                                          [1, 256, 32, 32]  

100%|██████████| 397/397 [04:27<00:00,  1.49it/s, loss=0.273]


12


...

Epoch: 21 
            Table Loss -- Train: 0.056 Test: 0.051
            Table Acc -- Train: 0.982 Test: 0.983
            Table F1 -- Train: 0.628 Test: 0.911
            Table Precision -- Train: 0.628 Test: 0.929
            Table Recall -- Train: 0.653 Test: 0.902
            
            Col Loss -- Train: 0.000 Test: 0.000
            Col Acc -- Train: 1.000 Test: 1.000
            Col F1 -- Train: 0.000 Test: 0.000
            Col Precision -- Train: 0.000 Test: 0.000
            Col Recall -- Train: 0.000 Test: 0.000
        
Checkpoint Saved at  densenet_config_4_model_checkpoint.pth.tar
11
in train_on_epoch


100%|██████████| 397/397 [01:39<00:00,  3.98it/s, loss=0.0413]


12


100%|██████████| 25/25 [00:14<00:00,  1.75it/s, loss=0.0567]

Epoch: 22 
            Table Loss -- Train: 0.041 Test: 0.057
            Table Acc -- Train: 0.986 Test: 0.981
            Table F1 -- Train: 0.668 Test: 0.893
            Table Precision -- Train: 0.669 Test: 0.888
            Table Recall -- Train: 0.686 Test: 0.907
            
            Col Loss -- Train: 0.000 Test: 0.000
            Col Acc -- Train: 1.000 Test: 1.000
            Col F1 -- Train: 0.000 Test: 0.000
            Col Precision -- Train: 0.000 Test: 0.000
            Col Recall -- Train: 0.000 Test: 0.000
        
Early Stopping
